In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('test')\
        .getOrCreate()

In [7]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [8]:
df_green.createOrReplaceTempView('green')

In [11]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1, 2
""")

In [10]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-02 14:00:00|  26| 327.4800000000001|            11|
|2020-01-06 15:00:00| 241|             184.7|             3|
|2020-01-22 13:00:00|  74| 666.2399999999999|            44|
|2020-01-14 23:00:00| 223|             48.31|             4|
|2020-01-09 23:00:00| 260|202.33000000000004|            16|
|2020-01-30 09:00:00|  75|1010.3499999999995|            67|
|2020-01-11 21:00:00|  41| 700.5999999999997|            50|
|2020-01-07 18:00:00|  45|112.39000000000001|             4|
|2020-01-02 09:00:00|  42| 555.7500000000001|            33|
|2020-01-04 23:00:00|  25|             376.4|            26|
|2020-01-25 11:00:00| 259|            161.21|             3|
|2020-01-16 20:00:00|  52|             108.6|             7|
|2020-01-16 06:00:00|  74| 600.7999999999998|            37|
|2020-01-17 07:00:00|  4

In [20]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [15]:
df_yellow.createOrReplaceTempView('yellow')

In [16]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [17]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-16 18:00:00| 232|261.71000000000004|            12|
|2020-01-06 05:00:00|  48|2105.5099999999984|           136|
|2020-01-03 15:00:00| 163| 5261.330000000002|           348|
|2020-01-06 11:00:00|  43|2731.4199999999983|           185|
|2020-01-17 07:00:00|  24|           1058.45|            62|
|2020-01-12 11:00:00| 163| 4039.869999999997|           244|
|2020-01-20 20:00:00| 144|1758.8999999999996|           107|
|2020-01-27 18:00:00| 162|11701.700000000033|           698|
|2020-01-26 03:00:00| 263|           1156.81|            69|
|2020-01-20 18:00:00| 113|2801.8799999999987|           197|
|2020-01-24 15:00:00| 186|7813.4200000000155|           446|
|2020-01-26 16:00:00| 236| 7387.040000000016|           509|
|2020-01-09 18:00:00| 263| 4200.289999999999|           268|
|2020-01-26 16:00:00| 23

In [21]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [25]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_of_records')

df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_of_records')

In [29]:
# what if we use the computed data
df_green_revenue_comp = spark.read.parquet('data/report/revenue/yellow')
df_yellow_revenue_comp = spark.read.parquet('data/report/revenue/green')


In [30]:
df_yellow_revenue_tmp = df_yellow_revenue_comp \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_of_records')

df_green_revenue_tmp = df_green_revenue_comp \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_of_records')

In [31]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [32]:
df_join.show()

+-------------------+----+------------------+-----------------------+------------------+------------------------+
|               hour|zone|      green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|
+-------------------+----+------------------+-----------------------+------------------+------------------------+
|2020-01-01 00:00:00|  15|             34.09|                      1|              null|                    null|
|2020-01-01 00:00:00|  25|            324.35|                     16|             531.0|                      26|
|2020-01-01 00:00:00|  29|              null|                   null|              61.3|                       1|
|2020-01-01 00:00:00|  33|            255.56|                      8| 317.2700000000001|                      11|
|2020-01-01 00:00:00|  34|              19.3|                      1|              null|                    null|
|2020-01-01 00:00:00|  41|1256.5300000000002|                     80|1363.9599999999982|

In [34]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [36]:
df_zones = spark.read.parquet('zones/')

In [37]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [38]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [44]:
df_result.drop('LocationID', 'zone').write.parquet('data/report/revenue/zones', mode='overwrite')